Install Requirements

In [3]:
!pip install -r requirements.txt

Open CSV

In [4]:
import csv

# Ruta del archivo CSV
archivo_csv = 'data/resume.csv'

# Lista para almacenar los datos del CSV
datos = []

# Abrir el archivo CSV y leer los datos
with open(archivo_csv, newline='') as csvfile:
    lector_csv = csv.reader(csvfile, delimiter=',')
    # Iterar sobre cada fila del CSV
    for fila in lector_csv:
        datos.append(fila)

In [21]:
len(datos)

2485

In [45]:
def chunk_by_words(text, chunk_size=300, overlap=10):
    tokens = text.split()
    chunks = []
    i = 0
    while i < len(tokens):
        chunks.append(" ".join(tokens[i:i+chunk_size]))
        i += chunk_size - overlap
    return chunks

In [46]:
speech = "Nosotros elegimos ir a la luna. Elegimos ir a la luna. Elegimos ir a la luna en esta década y hacer otras cosas no porque sean fáciles, sino porque son difíciles. Porque esa meta servirá para organizar y medir lo mejor de nuestras energías y habilidades, porque ese desafío es uno que estamos dispuestos a aceptar."

chunks = chunk_by_words(speech, chunk_size=20, overlap=5)
for idx, chunk in enumerate(chunks):
    print(f"Chunk {idx}: {chunk}\n")
    print(f"Chunk {idx}: {chunk}\n")

Chunk 0: Nosotros elegimos ir a la luna. Elegimos ir a la luna. Elegimos ir a la luna en esta década y

Chunk 0: Nosotros elegimos ir a la luna. Elegimos ir a la luna. Elegimos ir a la luna en esta década y

Chunk 1: luna en esta década y hacer otras cosas no porque sean fáciles, sino porque son difíciles. Porque esa meta servirá

Chunk 1: luna en esta década y hacer otras cosas no porque sean fáciles, sino porque son difíciles. Porque esa meta servirá

Chunk 2: difíciles. Porque esa meta servirá para organizar y medir lo mejor de nuestras energías y habilidades, porque ese desafío es

Chunk 2: difíciles. Porque esa meta servirá para organizar y medir lo mejor de nuestras energías y habilidades, porque ese desafío es

Chunk 3: habilidades, porque ese desafío es uno que estamos dispuestos a aceptar.

Chunk 3: habilidades, porque ese desafío es uno que estamos dispuestos a aceptar.



In [51]:
chunked_documents = []
chunk_unique_id = 0
for doc_id, document in enumerate(datos):
    chunks = chunk_by_words(document[1])
    for chunk_id, chunk in enumerate(chunks):
        chunked_documents.append({
            "id": chunk_unique_id,
            "document_id": doc_id,
            "text": chunk
        })
        chunk_unique_id += 1

In [52]:
print(len(chunked_documents))

8218


In [61]:
chunked_documents[1]

{'id': 1,
 'document_id': 1,
 'text': "HR ADMINISTRATOR/MARKETING ASSOCIATE HR ADMINISTRATOR Summary Dedicated Customer Service Manager with 15+ years of experience in Hospitality and Customer Service Management. Respected builder and leader of customer-focused teams; strives to instill a shared, enthusiastic commitment to customer service. Highlights Focused on customer satisfaction Team management Marketing savvy Conflict resolution techniques Training and development Skilled multi-tasker Client relations specialist Accomplishments Missouri DOT Supervisor Training Certification Certified by IHG in Customer Loyalty and Marketing by Segment Hilton Worldwide General Manager Training Certification Accomplished Trainer for cross server hospitality systems such as Hilton OnQ , Micros Opera PMS , Fidelio OPERA Reservation System (ORS) , Holidex Completed courses and seminars in customer service, sales strategies, inventory control, loss prevention, safety, time management, leadership and pe

In [55]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

In [56]:
def get_embedding(text):
    return model.encode(text)

In [58]:
from annoy import AnnoyIndex
import os

embedding_size = 768
index_name = "full-index-mananeras.ann"

In [59]:
chunk_id_mapping = {}
for chunk in chunked_documents:
    chunk_id_mapping[chunk["id"]] = chunk

In [63]:
if not os.path.exists(index_name):
    index = AnnoyIndex(embedding_size, 'angular')

    for idx, chunk in chunk_id_mapping.items():

        v = get_embedding(chunk["text"])

        index.add_item(idx, v)

    index.build(10)
    index.save(index_name)

In [64]:
index = AnnoyIndex(embedding_size, 'angular')
index.load(index_name)

True

In [65]:
pregunta = "¿which summarizes fits the following requirement? Senior Software Developer  Responsibilities:  Develop web applications using Java and React. Design and maintain robust APIs. Collaborate on technical design and software architecture. Optimize performance and resolve technical issues. Document code and processes. Requirements:  3+ years of software development experience. Proficiency in Java and React. Experience with RESTful APIs. Strong analytical and problem-solving skills. Communication skills in Spanish/English. Desirable:  Experience with agile methodologies. Knowledge of databases and Git."

In [67]:
print(pregunta)
embedding_pregunta = get_embedding(pregunta)
#print(embedding_pregunta)

¿which summarizes fits the following requirement? Senior Software Developer  Responsibilities:  Develop web applications using Java and React. Design and maintain robust APIs. Collaborate on technical design and software architecture. Optimize performance and resolve technical issues. Document code and processes. Requirements:  3+ years of software development experience. Proficiency in Java and React. Experience with RESTful APIs. Strong analytical and problem-solving skills. Communication skills in Spanish/English. Desirable:  Experience with agile methodologies. Knowledge of databases and Git.


In [68]:
ids_potenciales_respuestas = index.get_nns_by_vector(embedding_pregunta, 5)
ids_potenciales_respuestas

[4140, 3902, 3075, 409, 5940]

In [76]:
potenciales_respuestas = [chunk_id_mapping[idx] for idx in ids_potenciales_respuestas]
texto_potencial = [chunk["text"] for chunk in potenciales_respuestas]
texto_potencial

['SOFTWARE ENGINEER Summary Accomplished development professional with over ten years of experience in integrating IT Operations processes with sustainable customized applications to promote team performance and efficiency gains. An apt student in programming and markup languages matching multiple technologies together to best enhance and complement IT Service Management and Project Management best practices and deliver consistent, quality solutions in the fast-paced, evolving environment of technology. Highlights Languages: HTML, JavaScript, CSS, Perl, XML, XSLT, JSON, PHP, mySQL, VBScript Frameworks: AngularJS, Bootstrap Certifications: ITIL Foundation, Service-Now Application Developer Applications/Platforms: Service-Now, Salesforce, Confluence, JIRA, Rally, Gomez, SiteScope, Keynote, Omniture, Urchin, Request Tracker Accomplishments Launched Incident Management, Problem Management and Change Management tied to a centralized Configuration Management Database, leveraging Service-Now 

Create OpenAI method to get answers

In [72]:
from openai import OpenAI

client = OpenAI()

def query_llm(prompt, model="gpt-3.5-turbo"):
    completions = client.chat.completions.create(

    model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ],
        temperature=0.0,
    )

    return completions.choices[0].message.content

In [71]:
def get_answer(question, context_documents):
    context_divider = "\n---\n"
    question = question.strip()

    context = context_divider.join(context_documents)

    template_rag = """You are an assistant in the Human Resources department of an outsourcing company.
Respond to the question based on the context required by the employee.
The context is delimited by the inverted commas.

```
{context}
```

Question: {question}
"""
    final_prompt = template_rag.format(context=context, question=question)
    return query_llm(final_prompt)

In [74]:
print(pregunta)
print()
print(get_answer(pregunta, texto_potencial))

¿which summarizes fits the following requirement? Senior Software Developer  Responsibilities:  Develop web applications using Java and React. Design and maintain robust APIs. Collaborate on technical design and software architecture. Optimize performance and resolve technical issues. Document code and processes. Requirements:  3+ years of software development experience. Proficiency in Java and React. Experience with RESTful APIs. Strong analytical and problem-solving skills. Communication skills in Spanish/English. Desirable:  Experience with agile methodologies. Knowledge of databases and Git.



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}